In [1]:
import cv2
import numpy as np
import math as m

def bilinear_interpolation(img, x, y):
    h,w = int(y), int(x)                                            ## 소수점의 위치좌표 x,y를 int형으로 cast하여 소수점 없애기
    b = x-w                                                         ## x축에서 x, x+1 좌표간의 거리 비율
    a = y-h                                                         ## y축에서 y, y+1 좌표간의 거리 비율
    
    X1 = np.dot(1-b,img[h,w]) + np.dot(b,img[h,w+1])                ## bilinear interpolation 식 구현
    X2 = np.dot(1-b,img[h+1,w]) + np.dot(b,img[h+1,w+1])
    X = np.dot(1-a,X1) + np.dot(a,X2)

    return X

def Transform(img, trans_M) :
    
    img_r = np.zeros((height,width,3), np.uint8)                    ## 원본과 같은 사이즈의 결과이미지 초기화

    for h in range(0, img_r.shape[0]):                              ## 사이즈만큼 픽셀의 위치 변환(Transform)
        for w in range(0, img_r.shape[1]):
            ## 이미지를 Transform하면 window에서 벗어나는 pixel은 표현할 수 없기 때문에 벗어나는 pixel들은 0으로 값 대입 
            x, y = np.dot(trans_M, np.array([w,h,1]))[0:2]
            if x >= img.shape[1]-1 or y >= img.shape[0]-1 or x < 0 or y < 0:  
                img_r[h,w] = [0,0,0]
            ## (640,480)범위 내에 찍히는 것들은 bi-linear로 interpolation한다.
            else :
                img_r[h,w] = bilinear_interpolation(img, x, y)
    
    return img_r

if __name__ == '__main__': 
    img = cv2.imread('../picture/mumbai.jpg')
    img = cv2.resize(img, (640,480))
    
    global height, width
    height, width = img.shape[0:2]
    
    th = m.radians(45)                                                                    ## theta 45도 값을 radian으로 설정
    m1 = np.array([[1, 0, width/2], [0, 1, height/2], [0,0,1]])                         
    m2 = np.array([[1, 0, -width/2], [0, 1, -height/2], [0,0,1]])                   
    m3 = np.array([[m.cos(th), -m.sin(th), 0], [m.sin(th), m.cos(th), 0], [0,0,1]]) 
    
    rotation = np.dot(m1, np.dot(m3,m2))                                                 ## Rotation
    scaling = np.array([[1/2,0,0], [0,1/2,0], [0,0,1]])                                  ## Scaling (이미지 2배 축소)
    translation = np.array([[1,0,160], [0,1,120], [0,0,1]])                              ## Translation
    
    ## Scaling - Rotation - Translation 순으로 matrix 합성
    m_inv = np.linalg.inv(np.dot(translation,(np.dot(scaling,rotation))))
    img_r = Transform(img, m_inv)
    
    cv2.imshow("img", img)
    cv2.imshow("result", img_r)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [ ]:
%reset

In [3]:
x,y,w = np.dot(m_inv, [0,0,1])